In [1]:
import openai
import pandas as pd
import re

# OpenAI Client
client = openai.OpenAI()

# Define the GPT Prompt for Adversarial QA
prompt = """
Generate a list of 50 meaningful (Cause, Effect) pairs tailored to adversarial question answering.
The pairs should involve:
- Multi-hop inference (e.g., combining multiple facts)
- Counterfactual reasoning (e.g., what would have happened if...)
- Indirect causal relationships (e.g., cause is implied, not explicitly stated)
- Comparative reasoning (e.g., A is bigger than B, so X happens)
- Requires external knowledge (e.g., historical or scientific reasoning)

Each pair should be formatted as: (Cause, Effect)

Example:
(Failure to evacuate before a hurricane, Severe loss of life)
(A major blackout in a city, Increased crime rates overnight)
(A nation's economy collapsing, Increased global market instability)
(Loss of internet connectivity in a region, Disruptions in remote work)
(Not following pandemic guidelines, Increased spread of infection)

Now generate 50 more such challenging (Cause, Effect) pairs:
"""

# Call GPT to generate causal pairs
try:
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Extract response content
    gpt_output = response.choices[0].message.content.strip()

    # Extracting cause-effect pairs using regex
    causal_pairs = []
    for match in re.findall(r"\(([^,]+),\s*([^,]+)\)", gpt_output):
        cause, effect = match
        causal_pairs.append((cause.strip(), effect.strip()))

    # If API fails, use a fallback list
    if not causal_pairs:
        print("⚠️ GPT returned an empty response. Using fallback data.")
        causal_pairs = [
            ("A solar flare disrupts communications", "GPS systems fail globally"),
            ("A cyber attack on a banking system", "Millions unable to access their money"),
            ("A bridge collapse in a major city", "Massive traffic congestion for weeks"),
            ("A misinformation campaign spreads online", "Public distrust in media increases"),
            ("Power grid failure during a heatwave", "Thousands suffer heat-related illnesses"),
            ("A critical security flaw is exposed", "Mass data breaches occur worldwide"),
            ("Unprecedented climate changes", "Displacement of millions due to natural disasters"),
            ("Trade war escalations", "Significant fluctuations in stock markets"),
            ("Breakthrough in quantum computing", "Standard encryption methods become obsolete"),
            ("Drought conditions persist", "Food prices skyrocket globally"),
            ("Unregulated AI development", "Unintended biases affect hiring decisions"),
            ("Space debris collides with a satellite", "Loss of GPS and communications"),
            ("Prolonged teacher strikes", "Decline in national education standards"),
            ("Increase in antibiotic resistance", "Previously treatable diseases become deadly"),
            ("Financial institutions collapse", "Recession spreads to multiple countries"),
            ("A popular social media platform goes offline", "Users migrate to alternative platforms"),
            ("Widespread deforestation", "Loss of biodiversity in rainforests"),
            ("Medical misinformation spreads", "Patients refuse life-saving treatments"),
            ("Rapid advancement in autonomous vehicles", "Reduced demand for traditional driving jobs"),
            ("Major oil spill in the ocean", "Severe disruption to marine ecosystems"),
        ]

    # Convert to DataFrame
    df = pd.DataFrame(causal_pairs, columns=["Cause", "Effect"])
    
    # Save to CSV
    output_path = "adversarialQA_generated_causal_pairs.csv"
    df.to_csv(output_path, index=False)

    print(f"\n✅ Successfully generated {len(df)} adversarial causal pairs and saved to {output_path}")

except Exception as e:
    print(f"❌ Error generating causal pairs: {e}")



✅ Successfully generated 50 adversarial causal pairs and saved to adversarialQA_generated_causal_pairs.csv


In [3]:
from datasets import load_dataset
import datasets
import re

# Suppress Hugging Face logs
datasets.utils.logging.set_verbosity_error()

# Load Adversarial QA dataset (specifying the config "adversarialQA")
dataset_path = "/home/hao/colm/datasets/AdversarialQA"
dataset = load_dataset("UCLNLP/adversarial_qa", "adversarialQA", cache_dir=dataset_path)

# Choose split
split = "train"

# Debug: Print a sample entry to check structure
if split in dataset:
    print("Sample entry structure:", dataset[split][0])

def preprocess_adversarialqa(dataset, split):
    """Extract and clean questions and corresponding answers from Adversarial QA dataset."""
    if split not in dataset:
        raise ValueError(f"Error: Split '{split}' not found. Available splits: {list(dataset.keys())}")
    
    processed_data = []
    for item in dataset[split]:
        question = item.get("question", "").strip()
        answers = item.get("answers", {}).get("text", [])  # Extract list of possible answers
        
        # Validate question and answers
        if question and answers:
            correct_answer = answers[0]  # Take the first answer as the correct one
            processed_data.append({"question": question, "answer": correct_answer})
    
    if not processed_data:
        raise ValueError("Error: No valid 'question' and 'answer' fields found in the dataset.")
    
    # Text cleaning
    for item in processed_data:
        item["question"] = re.sub(r"[^a-zA-Z0-9.,!? ]", "", item["question"]).strip()
        item["answer"] = re.sub(r"[^a-zA-Z0-9.,!? ]", "", item["answer"]).strip()
    
    return processed_data

# Process the text data
try:
    processed_samples = preprocess_adversarialqa(dataset, split)
    print(f"Processed {len(processed_samples)} question-answer pairs.")
    
    # Print a few samples
    for i, sample in enumerate(processed_samples[:5]):
        print(f"Sample {i+1}:")
        print(f"Question: {sample['question']}")
        print(f"Answer: {sample['answer']}")
        print("-")
except Exception as e:
    print(f"An error occurred: {e}")


Generating test split: 100%|██████████| 3000/3000 [00:00<00:00, 593225.78 examples/s]


Sample entry structure: {'id': '7ba1e8f4261d3170fcf42e84a81dd749116fae95', 'title': 'Brain', 'context': 'Another approach to brain function is to examine the consequences of damage to specific brain areas. Even though it is protected by the skull and meninges, surrounded by cerebrospinal fluid, and isolated from the bloodstream by the blood–brain barrier, the delicate nature of the brain makes it vulnerable to numerous diseases and several types of damage. In humans, the effects of strokes and other types of brain damage have been a key source of information about brain function. Because there is no ability to experimentally control the nature of the damage, however, this information is often difficult to interpret. In animal studies, most commonly involving rats, it is possible to use electrodes or locally injected chemicals to produce precise patterns of damage and then examine the consequences for behavior.', 'question': 'What sare the benifts of the blood brain barrir?', 'answers':